In [ ]:
# Instalando e carregando os pacotes
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS, PooledOLS
import statsmodels.api as sm

# Carregar base
base = pd.read_stata("Base1.dta")

# Resumo inicial da base
print(base.info())
print(base.describe())

# Verificando porcentagem de NA em UNION por período
na_stats = base.groupby('TIME')['UNION'].apply(lambda x: x.isna().mean() * 100)
print(na_stats)

# Filtrando períodos com TIME > 7
base = base[base['TIME'] > 7].copy()

# Criando variável categórica para UNION
base['UNION_F'] = pd.Categorical(
    np.where(base['UNION'].isna(), "Desconhecido",
             np.where(base['UNION'] == 1, "Sim", "Não")),
    categories=["Não", "Sim", "Desconhecido"],
    ordered=True
)

# Checando NAs
print("NAs em EARNINGS:", base['EARNINGS'].isna().sum())
print("NAs em UNION:", base['UNION'].isna().sum())

# Definindo painel
# Criando as duplicatas das colunas 'ID' e 'TIME'
base['ID_DUPLICATE'] = base['ID']
base['TIME_DUPLICATE'] = base['TIME']

# Usando as duplicatas para definir o índice
base = base.set_index(['ID_DUPLICATE', 'TIME_DUPLICATE']).sort_index()

# Verificando se há duplicatas no índice
print(base.index.duplicated().sum())

# Se houver duplicatas, podemos remover as duplicatas ou ajustar
base = base[~base.index.duplicated(keep='first')]

# Diferenças primeiras por ID
base['d_EARNINGS'] = base.groupby(level=0)['EARNINGS'].diff()
base['d_UNION'] = base.groupby(level=0)['UNION'].diff()

# Criando lag da variável TIME
# Reverter o MultiIndex para aplicar shift() na coluna 'TIME'
base_reset = base.reset_index()
base_reset['TIME_LAG'] = base_reset.groupby('ID_DUPLICATE')['TIME'].shift()

# Reaplicar o valor TIME_LAG ao DataFrame original
base['TIME_LAG'] = base_reset['TIME_LAG'].values  # Usando .values para garantir alinhamento

# Criando fator de tempo
base['fTIME'] = base.index.get_level_values('TIME').astype('category')

# Modelo 1: Diferença de ganhos explicada por diferença em sindicalização com efeitos fixos de tempo
base['intercept'] = 1  # Requerido por PanelOLS
model1 = PanelOLS.from_formula('d_EARNINGS ~ d_UNION + EntityEffects + TimeEffects', 
                              data=base.dropna(subset=['d_EARNINGS', 'd_UNION']))
print(model1.fit(cov_type='robust').summary)

# Criar dummy D: 1 se d_EARNINGS ou d_UNION forem NA, 0 caso contrário
base['D'] = ((base['d_EARNINGS'].isna()) | (base['d_UNION'].isna())).astype(int)

# Modelos Pooled OLS para D
# Removendo NA em variáveis explicativas
model_data = base[['D', 'MALE', 'AGE', 'S']].dropna()

# Modelo r4: D ~ MALE
model_r4 = PooledOLS.from_formula('D ~ MALE', data=model_data)
print(model_r4.fit(cov_type='robust').summary)

# Modelo r5: D ~ MALE + AGE + S 
model_r5 = PooledOLS.from_formula('D ~ MALE + AGE + S', data=model_data)
print(model_r5.fit(cov_type='robust').summary)

# Modelo r6: D ~ MALE + AGE + S + fator de tempo
model_data['TIME'] = model_data.index.get_level_values('TIME')
model_r6 = PooledOLS.from_formula('D ~ MALE + AGE + S + C(TIME)', data=model_data)
print(model_r6.fit(cov_type='robust').summary)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42815 entries, 0 to 42814
Data columns (total 34 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TIME      42815 non-null  float32
 1   ID        42815 non-null  float32
 2   MALE      42815 non-null  float32
 3   AGE       42815 non-null  float32
 4   S         42815 non-null  float32
 5   ETHBLACK  42815 non-null  float32
 6   ETHHISP   42815 non-null  float32
 7   HEIGHT85  41898 non-null  float32
 8   WEIGHT    31067 non-null  float32
 9   ASVAB02   42815 non-null  float32
 10  ASVAB03   42815 non-null  float32
 11  ASVAB04   42815 non-null  float32
 12  ASVABC    42815 non-null  float32
 13  SM        40829 non-null  float32
 14  SF        39279 non-null  float32
 15  SIBLINGS  42779 non-null  float32
 16  YOUNGCH   18287 non-null  float32
 17  MARRIED   42813 non-null  float32
 18  SINGLE    42813 non-null  float32
 19  SOONMARR  36472 non-null  float32
 20  URBAN     42815 non-null  fl